In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import mahotas
import h5py
import imageio as iio
import os
from os import listdir
from itertools import chain
import pickle as pkl


In [2]:
def read_images(string):
    AllImages = []
    hue_features_all = []
    haralick_features_all = []
    for i in range(0,10):
        dirname = '../datasets/'+string+'/'+string+'/n'+str(i)
        images = []
        for image in os.listdir(dirname):
            img = iio.imread(dirname+'/' +image)

            image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            hue_features = cv2.HuMoments(cv2.moments(image_gray)).flatten()

            haralick_features = mahotas.features.haralick(image_gray).mean(axis=0)

            hue_features_all.append(hue_features)
            haralick_features_all.append(haralick_features)
            images.append(img)
        AllImages.append(images)
    return AllImages ,hue_features_all ,haralick_features_all


In [3]:
def histcaclulate(AllImages , rgb):   
    AllHists = []
    for images in AllImages:
        hists = []
        for image in images:
            hist = cv2.calcHist(image,[rgb],None,[256],[0,256]) 
            hists.append(hist)
        AllHists.append(hists)
    return AllHists

In [4]:
def concatenatehists(AllHists0,AllHists1,AllHists2):   
    Allhists = []
    for i in range(10):
        folderhists = []
        for j in range(len(AllHists0[i])):
            imagehist = np.concatenate([AllHists0[i][j] , AllHists1[i][j] , AllHists2[i][j]]) 
            folderhists.append(imagehist)
        Allhists.append(folderhists)
    return Allhists


In [5]:
def flator(lists):
    result = []
    for folder in lists:
        images = []
        for image in folder:
            images.append(image.ravel())
        result.append(images)
    return result
        

In [6]:
def findlabels(Allhists):
    lables = []
    for i in range(len(Allhists)):
        for j in range(len(Allhists[i])):
            lables.append(i)
    return lables


In [8]:
def handle(string):
    All = read_images(string)
    AllImages = All[0]
    hue_features_all = All[1]
    haralick_features_all = All[2]
    AllHists0 = histcaclulate(AllImages , 0)
    AllHists1 = histcaclulate(AllImages , 1)
    AllHists2 = histcaclulate(AllImages , 2)
    del All
    del AllImages
    Allhists = concatenatehists(AllHists0,AllHists1,AllHists2)   
    del AllHists0
    del AllHists1
    del AllHists2
    flatlist = flator(Allhists)
    lables = findlabels(Allhists)
    flatlist = list(chain.from_iterable(flatlist))
    all_featurs = np.concatenate((flatlist , hue_features_all, haralick_features_all), axis = 1)
    del flatlist
    return all_featurs , lables

In [9]:
X_train ,y_train = handle('training')

In [13]:
with open('X_train.pkl', 'wb') as f:
    pkl.dump(X_train, f)
with open('y_train.pkl', 'wb') as f:
    pkl.dump(y_train, f)

In [10]:
X_test ,y_test = handle('validation')

In [14]:
with open('X_test.pkl', 'wb') as f:
    pkl.dump(X_test, f)
with open('y_test.pkl', 'wb') as f:
    pkl.dump(y_test, f)